## Country-level INFRA-SAP

Create summary statistics at admin-2 and country-level

In [1]:
import os, sys, time, importlib

import geopandas as gpd
import pandas as pd
import networkx as nx
sys.path.append('/home/wb514197/Repos/GOSTnets')

import GOSTnets as gn
import rasterio as rio
from osgeo import gdal
import GOSTnets.calculate_od_raw as calcOD
import numpy as np

%load_ext autoreload
%autoreload 2

sys.path.append('/home/wb514197/Repos/INFRA_SAP')
from infrasap import aggregator
from shapely.wkt import loads
import infrasap.rai_calculator as rai

In [2]:
country = 'guatemala'
iso3 = 'GTM'
epsg = 26915

### Load data

In [3]:
base_in = "/home/public/Data/PROJECTS/INFRA_SAP"
in_folder = os.path.join(base_in, iso3)

# define data paths
focal_admin2 = os.path.join(in_folder, "admin.shp")
focal_osm = os.path.join(in_folder, f"{country}-latest.osm.pbf")
pop_name = "WP_2020_1km"
wp_1km = os.path.join(in_folder, f"{pop_name}.tif")
urban_extents = os.path.join(in_folder, "urban_extents.shp")
airports = os.path.join(in_folder, "airports.shp")
ports = os.path.join(in_folder, "ports.shp")
borders = os.path.join(in_folder, "borders.shp")
G_path = os.path.join(in_folder, 'graph', f"G_{iso3}.pickle")

base_out = "/home/wb514197/data/INFRA_SAP" # GOT permission denied using public 
out_folder = os.path.join(base_out, iso3)
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [4]:
tables_path = os.path.join(out_folder, 'tables')
if not os.path.exists(tables_path):
    os.mkdir(tables_path)

In [5]:
target = gpd.read_file(focal_admin2)

### 1. RAI

In [7]:
%%time
rai_roadnetwork = rai.extract_rai_network(focal_osm, epsg=epsg)

/home/wb514197/.conda/envs/geo/lib/python3.7/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


CPU times: user 28min 10s, sys: 12.2 s, total: 28min 22s
Wall time: 28min 20s


In [8]:
rai_roadnetwork = rai_roadnetwork.to_crs('EPSG:4326')

In [9]:
%%time
rai = rai.calculate_rai(target, "OBJECTID", rio.open(wp_1km), rai_roadnetwork, tables_path)

CPU times: user 13min 4s, sys: 3.87 s, total: 13min 8s
Wall time: 13min 8s


In [14]:
rai.head()

,OBJECTID,ISO_A2,WB_ADM1_CO,WB_ADM0_CO,WB_ADM0_NA,WB_ADM1_NA,WB_ADM2_CO,WB_ADM2_NA,ISO3,POP,RAI_POP_1,RAI_POP_2,RAI_POP_3,RAI_POP_4,RAI_Percent_1,RAI_Percent_2,RAI_Percent_3,RAI_Percent_4
0,17876,IQ,1564,118,Iraq,Anbar,36754,Ana,IRQ,66928.312500,5989.950195,8072.481934,11696.504883,14682.267578,0.089498,0.120614,0.174762,0.219373
1,17877,IQ,1564,118,Iraq,Anbar,36755,Falluja,IRQ,768829.250000,204171.515625,543152.062500,609002.375000,617388.937500,0.265562,0.706466,0.792117,0.803025
2,17878,IQ,1564,118,Iraq,Anbar,36756,Haditha,IRQ,120795.054688,30427.453125,75924.359375,85604.562500,89946.023438,0.251893,0.628539,0.708676,0.744617
3,17879,IQ,1564,118,Iraq,Anbar,36757,Heet,IRQ,210850.453125,43696.554688,109256.742188,141552.187500,145731.781250,0.207240,0.518172,0.671339,0.691162
4,17880,IQ,1564,118,Iraq,Anbar,36758,Qa'im,IRQ,190004.515625,43391.585938,84418.015625,118118.820312,126646.109375,0.228371,0.444295,0.621663,0.666543


In [15]:
rai['RAI_Percent_1'] = rai.RAI_POP_1/rai.POP
rai['RAI_Percent_2'] = rai.RAI_POP_2/rai.POP
rai['RAI_Percent_3'] = rai.RAI_POP_3/rai.POP
rai['RAI_Percent_4'] = rai.RAI_POP_4/rai.POP

In [18]:
# don't need this anymore
# rai = rai[['OBJECTID','POP','RAI_POP_1', 'RAI_POP_2', 'RAI_POP_3', 'RAI_POP_4',
#            'RAI_Percent_1', 'RAI_Percent_2', 'RAI_Percent_3', 'RAI_Percent_4']]
# rai.rename(columns={'ID':'OBJECTID'}, inplace=True)
# rai = target.merge(rai, on="OBJECTID")
# rai.drop(['geometry','Shape_Leng','Shape_Area'], axis=1, inplace=True)

In [21]:
rai.to_excel(os.path.join(tables_path, "1Rural Access to Roads.xlsx"), index=False)

### 2. Reload OD Matrix

In [22]:
OD = pd.read_csv(os.path.join(out_folder, 'OD_08_06.csv'), header=[0,1], index_col=0)

In [23]:
OD['city'] = OD['city'].apply(lambda x: (x/3600))
OD['port'] = OD['port'].apply(lambda x: (x/3600))
OD['border'] = OD['border'].apply(lambda x: (x/3600))
OD['airport'] = OD['airport'].apply(lambda x: (x/3600))

### 3. District-level summaries

**Percentage of rural population with access to a main city (>50k inhabitants) in XX hours**

In [24]:
dest_type='city'
rural=True
thresholds=[0,0.5,1,1.5,2,3,4,5,6,7,10,1000] # hours
rural_access = aggregator.calculate_access_percentages(OD, target, 'city', True, urban_extents, thresholds=thresholds)

In [25]:
rural_access.rename(columns = {
    '(10.0, 1000.0]':'> 10.0',
    '(10.0, 1000.0] pop':'> 10.0 pop'
}, inplace=True)
rural_access.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)

In [28]:
rural_access.to_excel(os.path.join(tables_path, "2Rural Access to Cities.xlsx"), index=False)

**Travel time to closest city**

In [29]:
city_tt = aggregator.calculate_access_percentages(OD, target, 'city', thresholds=thresholds)
city_tt.rename(columns = {
    '(10.0, 1000.0]':'> 10.0',
    '(10.0, 1000.0] pop':'> 10.0 pop'
}, inplace=True)
city_tt.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)
city_tt.to_excel(os.path.join(tables_path, "3Travel Time to Nearest City.xlsx"), index=False)

**Travel time to capital**

In [30]:
dest_all = pd.read_csv(os.path.join(out_folder, 'destination_all.csv'), index_col=0)
cap_idx = dest_all.sort_values('Pop', ascending=False).iloc[[0]].index[0]

In [31]:
capital_tt = aggregator.calculate_access_percentages(OD, target, 'city', thresholds=thresholds, capital=str(cap_idx))
capital_tt.rename(columns = {
    '(10.0, 1000.0]':'> 10.0',
    '(10.0, 1000.0] pop':'> 10.0 pop'
}, inplace=True)
capital_tt.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)
capital_tt.to_excel(os.path.join(tables_path, "4Travel Time to Capital.xlsx"), index=False)

**Time to drive to the closest land border point**

In [32]:
border_tt = aggregator.calculate_access_percentages(OD, target, 'border', thresholds=thresholds)
border_tt.rename(columns = {
    '(10.0, 1000.0]':'> 10.0',
    '(10.0, 1000.0] pop':'> 10.0 pop'
}, inplace=True)
border_tt.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)
border_tt.to_excel(os.path.join(tables_path, "6Travel Time to Nearest Border.xlsx"), index=False)

**Time to drive to the closest airport**

In [33]:
airport_tt = aggregator.calculate_access_percentages(OD, target, 'airport', thresholds=thresholds)
airport_tt.rename(columns = {
    '(10.0, 1000.0]':'> 10.0',
    '(10.0, 1000.0] pop':'> 10.0 pop'
}, inplace=True)
airport_tt.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)
airport_tt.to_excel(os.path.join(tables_path, "5Travel Time to Nearest Airport.xlsx"), index=False)

**Time to drive to the closest port**

In [34]:
port_tt = aggregator.calculate_access_percentages(OD, target, 'port', thresholds=thresholds)
port_tt.rename(columns = {
    '(10.0, 1000.0]':'> 10.0',
    '(10.0, 1000.0] pop':'> 10.0 pop'
}, inplace=True)
port_tt.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)
port_tt.to_excel(os.path.join(tables_path, "7Travel Time to Nearest Port.xlsx"), index=False)

#### Aggregate to the country-level

In [35]:
country_level = aggregator.agregate_to_country(rural_access, "Rural pop. with access to a main city")
country_level = aggregator.agregate_to_country(city_tt, "Travel time to closest city", country_level)
country_level = aggregator.agregate_to_country(capital_tt, "Travel time to capital", country_level)
country_level = aggregator.agregate_to_country(border_tt, "Travel time to closest border", country_level)
country_level = aggregator.agregate_to_country(airport_tt, "Travel time to closest airport", country_level)
country_level = aggregator.agregate_to_country(port_tt, "Travel time to closest port", country_level)

In [36]:
country_level

,"(0.0, 0.5] pop","(0.5, 1.0] pop","(1.0, 1.5] pop","(1.5, 2.0] pop","(2.0, 3.0] pop","(3.0, 4.0] pop","(4.0, 5.0] pop","(5.0, 6.0] pop","(6.0, 7.0] pop","(7.0, 10.0] pop",> 10.0 pop
Rural pop. with access to a main city,0.018014,0.062157,0.098691,0.101461,0.123199,0.063264,0.063695,0.072165,0.059148,0.214519,0.123688
Travel time to closest city,0.029020,0.069692,0.102991,0.103535,0.120851,0.061099,0.061235,0.069465,0.056864,0.206339,0.118910
Travel time to capital,0.000837,0.002398,0.004896,0.006746,0.020779,0.036535,0.045540,0.059925,0.064593,0.257982,0.499768
Travel time to closest border,0.004072,0.010110,0.014350,0.015868,0.046428,0.060697,0.073859,0.104934,0.111566,0.233163,0.324953
Travel time to closest airport,0.007470,0.020523,0.030906,0.043468,0.095560,0.108207,0.124322,0.122866,0.086635,0.206359,0.153684
Travel time to closest port,0.004703,0.007840,0.008413,0.009203,0.020247,0.025605,0.035806,0.043710,0.036691,0.114169,0.693613


#### Pop-weighted average country-level

In [38]:
raster_path = wp_1km

In [39]:
national_average = aggregator.pop_weighted_average_national(os.path.join(out_folder,'travel_time','cities_min_tt.tif'), raster_path, 'Travel time to closest city')
national_average = aggregator.pop_weighted_average_national(os.path.join(out_folder,'travel_time','capital_tt.tif'), raster_path, 'Travel time to capital', national_average)
national_average = aggregator.pop_weighted_average_national(os.path.join(out_folder,'travel_time','borders_min_tt.tif'), raster_path, 'Travel time to closest border', national_average)
national_average = aggregator.pop_weighted_average_national(os.path.join(out_folder,'travel_time','airport_min_tt.tif'), raster_path, 'Travel time to closest airport', national_average)
national_average = aggregator.pop_weighted_average_national(os.path.join(out_folder,'travel_time','port_min_tt.tif'), raster_path, 'Travel time to closest port', national_average)

In [40]:
country_level = country_level.join(national_average)

In [41]:
country_level.rename(columns = {
    '(0.0, 0.5] pop':'(0.0, 0.5]',
    '(0.5, 1.0] pop':'(0.5, 1.0]',
    '(1.0, 1.5] pop':'(1.0, 1.5]',
    '(1.5, 2.0] pop':'(1.5, 2.0]',
    '(2.0, 3.0] pop':'(2.0, 3.0]',
    '(3.0, 4.0] pop':'(3.0, 4.0]',
    '(4.0, 5.0] pop':'(4.0, 5.0]',
    '(5.0, 6.0] pop':'(5.0, 6.0]',
    '(6.0, 7.0] pop':'(6.0, 7.0]',
    '(7.0, 10.0] pop':'(7.0, 10.0]',
    '> 10.0 pop':'> 10.0'  
}, inplace=True)
country_level.to_excel(os.path.join(tables_path, "9National Level Aggregates.xlsx"), index=False)

#### Pop-weighted average by district

In [42]:
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_time','cities_min_tt.tif'), raster_path, 'tt_min_city')
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_time','capital_tt.tif'), raster_path, 'tt_capital', table=weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_time','borders_min_tt.tif'), raster_path, 'tt_border', table=weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_time','airport_min_tt.tif'), raster_path, 'tt_airport', table=weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_time','port_min_tt.tif'), raster_path, 'tt_port', table=weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_distance','cities_min_dist.tif'), raster_path, 'dist_cities', weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_distance','airport_min_dist.tif'), raster_path, 'dist_airport', weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_distance','port_min_dist.tif'), raster_path, 'dist_ports', weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_distance','borders_min_dist.tif'), raster_path, 'dist_borders', weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_distance','capital_dist.tif'), raster_path, 'dist_capital',weighted_average)

#### Market Access

In [43]:
out_access = os.path.join(out_folder, 'access')

In [44]:
ma5 = aggregator.pop_weighted_average(target, os.path.join(out_access,f"access_cities_d5.tif"), raster_path, 'ma_5')
ma6 = aggregator.pop_weighted_average(target, os.path.join(out_access,f"access_cities_d6.tif"), raster_path, 'ma_6')
ma7 = aggregator.pop_weighted_average(target, os.path.join(out_access,f"access_cities_d7.tif"), raster_path, 'ma_7')
ma8 = aggregator.pop_weighted_average(target, os.path.join(out_access,f"access_cities_d8.tif"), raster_path, 'ma_8')
ma9 = aggregator.pop_weighted_average(target, os.path.join(out_access,f"access_cities_d9.tif"), raster_path, 'ma_9')

In [45]:
ma = pd.concat([ma5, ma6[['ma_6']], ma7[['ma_7']], ma8[['ma_8']], ma9[['ma_9']]], axis=1)

#### Combine district-level data

In [46]:
district_summary = weighted_average.merge(ma[['OBJECTID','ma_5','ma_6','ma_7','ma_8','ma_9']], on="OBJECTID")

In [47]:
# rai = pd.read_excel(os.path.join(tables_path, '1Rural Access to Roads.xlsx'))
rai = rai[['OBJECTID','RAI_Percent_1','RAI_Percent_2', 'RAI_Percent_3', 'RAI_Percent_4']]
district_summary = district_summary.merge(rai, on='OBJECTID')

In [50]:
hubs = pd.read_excel(os.path.join(tables_path, '8Travel Time to Regional Hub.xlsx'))
hubs = hubs.rename(columns={'Time (hrs)':'tt_regional', 'Name1':'hub_name'}) # 'Regional Hub Name'
hubs = hubs[['OBJECTID','tt_regional','hub_name']]
district_summary = district_summary.merge(hubs, on='OBJECTID')

In [51]:
district_summary.to_file(os.path.join(out_folder,'admin_join.shp'), driver='ESRI Shapefile')

In [52]:
district_table = district_summary.copy()
district_table.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)
district_table.to_excel(os.path.join(tables_path, "10District Level Summary.xlsx"), index=False)